In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
file_path = '/Users/kanaha/statistics/SSDSE-D-2023.xlsx'
df = pd.read_excel(file_path)
print(df.head())

  SSDSE-D-2023   2021年 Prefecture         MA00           MB00  MB01 MB011  \
0         男女の別   地域コード       都道府県  推定人口（10歳以上）  学習・自己啓発・訓練の総数   外国語    英語   
1         0_総数  R00000         全国       112462           39.6  14.1  12.8   
2         0_総数  R01000        北海道         4614             35  11.1   9.9   
3         0_総数  R02000        青森県         1090           25.4   7.4   6.6   
4         0_総数  R03000        岩手県         1075           28.9   7.7   7.1   

      MB012             MB02        MB021  ... MG20   MG51 MG52 MG53  \
0  英語以外の外国語  商業実務・ビジネス関係(総数)  パソコンなどの情報処理  ...  その他  通勤・通学   仕事   学業   
1       4.2             20.1         16.5  ...   16     79  340  393   
2       3.3               17         14.1  ...   15     64  343  391   
3         2             12.7         10.3  ...   12     61  370  423   
4       2.4             12.8         10.3  ...   17     63  362  472   

       MH01      MH02      MH03      MH04      MH51       MH52  
0        起床      朝食開始      夕食開始        

In [3]:
# Excelファイルを読み込む際に、ヘッダーを2行目（インデックス1）にする
df = pd.read_excel(file_path, header=1)
#２列目を削除する
df = df.drop(df.columns[1], axis=1)

print(df.head())

   男女の別 都道府県  推定人口（10歳以上）  学習・自己啓発・訓練の総数   外国語    英語  英語以外の外国語  \
0  0_総数   全国       112462           39.6  14.1  12.8       4.2   
1  0_総数  北海道         4614           35.0  11.1   9.9       3.3   
2  0_総数  青森県         1090           25.4   7.4   6.6       2.0   
3  0_総数  岩手県         1075           28.9   7.7   7.1       2.4   
4  0_総数  宮城県         2065           37.5  12.4  10.9       4.0   

   商業実務・ビジネス関係(総数)  パソコンなどの情報処理  商業実務・ビジネス関係  ...  その他.4  通勤・通学.1  仕事.1  学業.1  \
0             20.1         16.5          8.5  ...     16       79   340   393   
1             17.0         14.1          6.7  ...     15       64   343   391   
2             12.7         10.3          5.2  ...     12       61   370   423   
3             12.8         10.3          5.4  ...     17       63   362   472   
4             18.9         16.0          7.4  ...     16       73   352   395   

         起床      朝食開始      夕食開始        就寝        出勤  仕事からの帰宅時間  
0  06:38:00  07:11:00  18:58:00  23:04:00  08:20:00

In [4]:
plt.rcParams['font.family'] = 'Hiragino Sans GB' # Macの場合の例
plt.rcParams['axes.unicode_minus'] = False # マイナス記号を正しく表示するため

##  趣味・娯楽における男女格差の要因分析

In [6]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# データDの読み込み（仮定）
# 実際には、ご自身のファイルパスに合わせて修正してください
file_path = 'SSDSE-D-2023.xlsx'
df_raw = pd.read_excel(file_path, header=1)

# 2列目（地域コード）を削除
df_raw = df_raw.drop(df_raw.columns[1], axis=1)

# 男女別のデータに分割
df_male = df_raw[df_raw['男女の別'] == '1_男'].copy()
df_female = df_raw[df_raw['男女の別'] == '2_女'].copy()

# 都道府県名をキーとして結合するためにインデックスを設定
df_male.set_index('都道府県', inplace=True)
df_female.set_index('都道府県', inplace=True)

# 目的変数と説明変数を作成するためのデータフレームを初期化
df_diff = pd.DataFrame(index=df_male.index)

# 目的変数: 趣味・娯楽の男女格差
df_diff['趣味・娯楽の格差'] = df_female['趣味・娯楽の総数'] - df_male['趣味・娯楽の総数']

# 説明変数: 各活動の男女格差
df_diff['スポーツの格差'] = df_female['スポーツの総数'] - df_male['スポーツの総数']
df_diff['学習・自己啓発の格差'] = df_female['学習・自己啓発・訓練の総数'] - df_male['学習・自己啓発・訓練の総数']
df_diff['ボランティアの格差'] = df_female['ボランティア活動の総数'] - df_male['ボランティア活動の総数']
df_diff['睡眠時間の格差'] = df_female['睡眠'] - df_male['睡眠']

# 推定人口を説明変数として追加
# 男女の合計値として、男性の推定人口を使用（女性もほぼ同じ値のため）
df_diff['推定人口'] = df_male['推定人口（10歳以上）']

# 重回帰分析の実行
# 目的変数
y = df_diff['趣味・娯楽の格差']

# 説明変数
X = df_diff[['スポーツの格差', '学習・自己啓発の格差', 'ボランティアの格差', '睡眠時間の格差', '推定人口']]

# 定数項の追加（切片）
X = sm.add_constant(X)

# モデルの構築とフィッティング
model = sm.OLS(y, X)
results = model.fit()

# 結果の表示
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               趣味・娯楽の格差   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     2.593
Date:                Sat, 16 Aug 2025   Prob (F-statistic):             0.0393
Time:                        13:59:39   Log-Likelihood:                -60.892
No. Observations:                  48   AIC:                             133.8
Df Residuals:                      42   BIC:                             145.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2437      0.536     -0.455      0.6

1. モデル全体の評価
R-squared (決定係数): 0.236
これは、モデルが目的変数である「趣味・娯楽の格差」の変動の約23.6%を説明できていることを意味します。この値は高くないため、このモデルでは「趣味・娯楽の格差」を完全に説明できておらず、他に影響を与える要因があることを示唆しています。
F-statistic (F統計量): 2.593, Prob (F-statistic): 0.0393
このp値は0.05を下回っているため、このモデル全体は統計的に有意であると判断できます。つまり、すべての説明変数のうち、少なくとも1つは目的変数に有意な影響を与えていると言えます。
2. 個々の説明変数の評価
ボランティアの格差 (ボランティアの格差):
coef (係数): 0.1509
P>|t| (p値): 0.020
このp値は0.05を下回っており、統計的に有意です。係数がプラスであることから、「ボランティア活動における男女格差が大きい都道府県ほど、趣味・娯楽における男女格差も大きい」という正の相関関係があることが示されました。
スポーツの格差 (スポーツの格差):
coef (係数): 0.1105
P>|t| (p値): 0.096
p値は0.05をわずかに上回っていますが、一般的に0.10を下回る場合は「弱い有意性」があると考えられます。プラスの係数から、スポーツ活動の男女格差が大きいほど、趣味・娯楽の男女格差も大きい傾向にあることが示唆されます。
その他の変数:
学習・自己啓発の格差、睡眠時間の格差、推定人口のp値はすべて0.10を大きく上回っており、これらは統計的に有意ではないと判断されます。つまり、これらの変数と「趣味・娯楽の格差」との間には、このモデルにおいては明確な関係性を見出すことができませんでした。
3. 注意点
多重共線性 (Multicollinearity):
Notesの欄にThe condition number is large...This might indicate that there are strong multicollinearity...と記載されています。これは、説明変数間に高い相関がある可能性があることを示しています。多重共線性が存在すると、係数の解釈が不安定になることがあるため注意が必要です。
結論
この重回帰分析の結果から、都道府県ごとの「趣味・娯楽の男女格差」は、「ボランティア活動の男女格差」と正の関係があることが統計的に有意に示されました。また、「スポーツ活動の男女格差」も弱いながらも同様の関係性を示唆しています。一方で、「学習・自己啓発の格差」「睡眠時間の格差」「推定人口」は、「趣味・娯楽の格差」と統計的に有意な関係はないと結論付けられます。